In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery, VectorizableTextQuery, VectorizableImageBinaryQuery

# Load the environment variables with dotenv
from dotenv import load_dotenv
load_dotenv()
import os

service_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
key = os.getenv("AZURE_AI_SEARCH_API_KEY")
index_name = "vector-1717153805663" # replace this if you named your index differently

# Create a client
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

print('Using endpoint', service_endpoint)
print('Using index', index_name)

# Connect to storage account where images are stored
AZURE_STORAGE_ACCOUNT_NAME = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
AZURE_STORAGE_ACCOUNT_KEY = os.getenv("AZURE_STORAGE_ACCOUNT_KEY")


from azure.storage.blob import BlobServiceClient

# Create a blob service client
blob_service_client = BlobServiceClient(account_url=f"https://{AZURE_STORAGE_ACCOUNT_NAME}.blob.core.windows.net", credential=AZURE_STORAGE_ACCOUNT_KEY)



In [ ]:
#import requests
from azure.storage.blob import BlobServiceClient
from IPython.display import display, Image

# Create a blob service client
    blob_service_client = BlobServiceClient(account_url="https://rifiel-public.blob.core.windows.net", credential="<your-account-key>")

In [ ]:
# A helper function to show images
def show_image(blob_name):
    blob_client = blob_service_client.get_blob_client("images", blob_name)
    stream = blob_client.download_blob().readall()
    display(Image(stream, width=200))

In [ ]:
# Define a helper function to search
def search(query):
    r = search_client.search(
        vector_queries=[VectorizableTextQuery(text=query, k_nearest_neighbors=5, fields="image_vector")]
    )
    first = None
    for doc in r:
        score_threshold = 0.6
        if doc["@search.score"] < score_threshold:
            break
        if first is None:
            first = doc
        print(doc["@search.score"],doc["title"], doc["metadata_storage_path"])
        show_image(doc["title"])

In [ ]:
search("do you have any backpacks?")

In [ ]:
search("do you have any red backpacks?")

In [ ]:
search("I want to go to the mountains. What can you show me?")

In [ ]:
search("do you have something for animal lovers?")

In [ ]:
search("I'd love to go fishing!")

In [ ]:
search("What abot music?")